# Exploring the functionalities of Tensorflow funcs

## Importing libraries

In [127]:
import tensorflow as tf
from tensorflow import keras
from ipynb.fs.full.Useful_funcs import pre_model, data_pipeline
from IPython.display import display, Markdown
from sklearn.datasets import fetch_california_housing

## Loading dataset

In [128]:
housing = fetch_california_housing()

In [129]:
x_train, x_train_scaled, x_valid, x_valid_scaled, x_test, x_test_scaled, y_train, y_valid, y_test = data_pipeline(housing)

- First we will define a simple func to calcualte the cube of a no.

In [2]:
def cube(x):
    return x ** 3

- We can call this func with a python value such a an int or float or a tensor.

In [3]:
cube(2)

8

In [4]:
cube(tf.constant(2.))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

- Now use tf.function() to convert this func into a tensorflow func.

In [5]:
tf_cube = tf.function(cube)

In [6]:
tf_cube

- This func can be used in the same way as the python func and it will return the same values but as tensors.

In [7]:
tf_cube(2.)

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [8]:
tf_cube(tf.constant(2.))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

- Under the hood the TF func analyzes the computations performed by the python func and generates an equivalent computation graph.
- Alternatively we can also use tf.function as a decorator which is more common.

In [9]:
@tf.function
def tf_cube(x):
    return x ** 3

- The original python func is still available via the python_function() attribute

In [10]:
tf_cube.python_function(2.)

8.0

### TF Functions and concrete functions

In [11]:
concrete_function = tf_cube.get_concrete_function(tf.constant(2.))

In [12]:
concrete_function.graph

In [13]:
concrete_function(tf.constant(2.))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [14]:
concrete_function is tf_cube.get_concrete_function(2.)

False

## Exploring func definitions and graphs

In [15]:
concrete_function.graph

In [16]:
ops = concrete_function.graph.get_operations()

In [17]:
ops

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'pow/y' type=Const>,
 <tf.Operation 'pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

In [18]:
pow_op = ops[2]

In [19]:
pow_op.inputs

(<tf.Tensor 'x:0' shape=() dtype=float32>,
 <tf.Tensor 'pow/y:0' shape=() dtype=float32>)

In [20]:
pow_op.outputs

[<tf.Tensor 'pow:0' shape=() dtype=float32>]

In [21]:
concrete_function.graph.get_operation_by_name('x')

<tf.Operation 'x' type=Placeholder>

In [22]:
concrete_function.graph.get_tensor_by_name('Identity:0')

<tf.Tensor 'Identity:0' shape=() dtype=float32>

In [24]:
concrete_function.function_def.signature

name: "__inference_tf_cube_25"
input_arg {
  name: "x"
  type: DT_FLOAT
}
output_arg {
  name: "identity"
  type: DT_FLOAT
}

## AutoGraph and Tracing

In [29]:
@tf.function
def tf_cube(x):
    return x ** 3

In [30]:
tf_cube

In [31]:
tf_cube(tf.constant(2.))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [32]:
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [33]:
result = tf_cube(2)
result = tf_cube(3)
result = tf_cube(tf.constant([[1., 2.]])) # New shape: trace!
result = tf_cube(tf.constant([[3., 4.], [5., 6.]])) # New shape: trace!
result = tf_cube(tf.constant([[7., 8.], [9., 10.], [11., 12.]])) # no trace

- In the above cell we can see that as we pass arguments of different shapes, each computation requires tracing and creating a new node in the graph. The warning states that tracing is an expensive process and gives reasons for this happening and solutions.

In [40]:
@tf.function(input_signature = [tf.TensorSpec([None, 28, 28], tf.float32)]) # Specifying the shape and dtype of the input
def shrink(images):
    return images[:, ::2, ::2] # Drops half the rows and columns

In [44]:
pre_model()

In [45]:
img_batch_1 = tf.random.uniform(shape = [100, 28, 28]) # Creating sample batches
img_batch_2 = tf.random.uniform(shape = [50, 28, 28])
preprocessed_images = shrink(img_batch_1) # TF traces the func

In [46]:
preprocessed_images = shrink(img_batch_2) # Reuses the same concrete func

In [47]:
img_batch_3 = tf.random.uniform(shape = [2, 2, 2]) # Batch with different shape
preprocessed_images = shrink(img_batch_3)

ValueError: Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor(
[[[0.7413678  0.62854624]
  [0.01738465 0.3431449 ]]

 [[0.51063764 0.3777541 ]
  [0.07321596 0.02137029]]], shape=(2, 2, 2), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name=None))

## Using Autograph to capture control flow

In [48]:
@tf.function # TF func with a for loop to add 10
def add_10(x):
    for i in range(10):
        x += 1
    return x

In [49]:
add_10(tf.constant(5.))

<tf.Tensor: shape=(), dtype=float32, numpy=15.0>

In [51]:
add_10.get_concrete_function(tf.constant(5.)).graph.get_operations() 

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'add/y' type=Const>,
 <tf.Operation 'add' type=AddV2>,
 <tf.Operation 'add_1/y' type=Const>,
 <tf.Operation 'add_1' type=AddV2>,
 <tf.Operation 'add_2/y' type=Const>,
 <tf.Operation 'add_2' type=AddV2>,
 <tf.Operation 'add_3/y' type=Const>,
 <tf.Operation 'add_3' type=AddV2>,
 <tf.Operation 'add_4/y' type=Const>,
 <tf.Operation 'add_4' type=AddV2>,
 <tf.Operation 'add_5/y' type=Const>,
 <tf.Operation 'add_5' type=AddV2>,
 <tf.Operation 'add_6/y' type=Const>,
 <tf.Operation 'add_6' type=AddV2>,
 <tf.Operation 'add_7/y' type=Const>,
 <tf.Operation 'add_7' type=AddV2>,
 <tf.Operation 'add_8/y' type=Const>,
 <tf.Operation 'add_8' type=AddV2>,
 <tf.Operation 'add_9/y' type=Const>,
 <tf.Operation 'add_9' type=AddV2>,
 <tf.Operation 'Identity' type=Identity>]

In [56]:
@tf.function
def add_10(x):
    condition = lambda i, x : tf.less(i, 10) # Condition checking i<10
    body = lambda i, x : (tf.add(i, 1), tf.add(x, 1)) # Increasing the counter and adding 1 to x
    final_i, final_x = tf.while_loop(condition, body, [tf.constant(0), x]) # Implementing while loop
    return final_x

In [57]:
add_10(tf.constant(5.))

<tf.Tensor: shape=(), dtype=float32, numpy=15.0>

In [58]:
add_10.get_concrete_function(tf.constant(5.)).graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'Const' type=Const>,
 <tf.Operation 'while/maximum_iterations' type=Const>,
 <tf.Operation 'while/loop_counter' type=Const>,
 <tf.Operation 'while' type=StatelessWhile>,
 <tf.Operation 'Identity' type=Identity>]

In [73]:
@tf.function
def add_10(x):
    for i in tf.range(10): # Using tf.range instead of range
        x += 1
    return x

In [74]:
add_10.get_concrete_function(tf.constant(5.)).graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'range/start' type=Const>,
 <tf.Operation 'range/limit' type=Const>,
 <tf.Operation 'range/delta' type=Const>,
 <tf.Operation 'range' type=Range>,
 <tf.Operation 'sub' type=Sub>,
 <tf.Operation 'floordiv' type=FloorDiv>,
 <tf.Operation 'mod' type=FloorMod>,
 <tf.Operation 'zeros_like' type=Const>,
 <tf.Operation 'NotEqual' type=NotEqual>,
 <tf.Operation 'Cast' type=Cast>,
 <tf.Operation 'add' type=AddV2>,
 <tf.Operation 'zeros_like_1' type=Const>,
 <tf.Operation 'Maximum' type=Maximum>,
 <tf.Operation 'while/loop_counter' type=Const>,
 <tf.Operation 'while' type=StatelessWhile>,
 <tf.Operation 'Identity' type=Identity>]

## Handling variables and other resources in TF funcs

In [75]:
counter = tf.Variable(0.)

In [76]:
@tf.function
def increment(counter, c = 1):
    return counter.assign_add(c)

In [77]:
increment(counter)

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [78]:
increment(counter)

<tf.Tensor: shape=(), dtype=float32, numpy=2.0>

In [79]:
function_def = increment.get_concrete_function(counter).function_def

In [83]:
function_def.signature.input_arg[0]

name: "counter"
type: DT_RESOURCE

In [84]:
counter = tf.Variable(0)

In [85]:
@tf.function
def increment(c = 1):
    return counter.assign_add(c)

In [86]:
increment()

<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [87]:
increment()

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [88]:
function_def = increment.get_concrete_function().function_def

In [92]:
function_def.signature.input_arg[0]

name: "assignaddvariableop_resource"
type: DT_RESOURCE

In [93]:
class Counter:
    def __init__(self):
        self.counter = tf.Variable(0.)
    @tf.function
    def increment(self, c = 1):
        return self.counter.assign_add(c)

In [94]:
c = Counter()

In [95]:
c.increment()

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [96]:
c.increment()

<tf.Tensor: shape=(), dtype=float32, numpy=2.0>

In [97]:
@tf.function
def add_10(x):
    for i in tf.range(10):
        x += 1
    return x

In [101]:
tf.autograph.to_code(add_10.python_function)

"def tf__add(x):\n    with ag__.FunctionScope('add_10', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:\n        do_return = False\n        retval_ = ag__.UndefinedReturnValue()\n\n        def get_state():\n            return (x,)\n\n        def set_state(vars_):\n            nonlocal x\n            (x,) = vars_\n\n        def loop_body(itr):\n            nonlocal x\n            i = itr\n            x = ag__.ld(x)\n            x += 1\n        i = ag__.Undefined('i')\n        ag__.for_stmt(ag__.converted_call(ag__.ld(tf).range, (10,), None, fscope), None, loop_body, get_state, set_state, ('x',), {'iterate_names': 'i'})\n        try:\n            do_return = True\n            retval_ = ag__.ld(x)\n        except:\n            do_return = False\n            raise\n        return fscope.ret(retval_, do_return)\n"

In [111]:
def display_tf_code(func):
    if hasattr(func, 'python_function'):
        func = func.python_function
    code = tf.autograph.to_code(func)
    display(Markdown('```python\n{}\n```'.format(code)))

In [112]:
display_tf_code(add_10)

```python
def tf__add(x):
    with ag__.FunctionScope('add_10', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (x,)

        def set_state(vars_):
            nonlocal x
            (x,) = vars_

        def loop_body(itr):
            nonlocal x
            i = itr
            x = ag__.ld(x)
            x += 1
        i = ag__.Undefined('i')
        ag__.for_stmt(ag__.converted_call(ag__.ld(tf).range, (10,), None, fscope), None, loop_body, get_state, set_state, ('x',), {'iterate_names': 'i'})
        try:
            do_return = True
            retval_ = ag__.ld(x)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)

```

## TF funcs and Keras

In [188]:
def custom_mse(y_true, y_pred): # Custom loss func
    print('Tracing custom_mse()')
    return tf.reduce_mean(tf.square(y_pred - y_true))

In [189]:
def custom_mae(y_true, y_pred): # Custom metric func
    print('Tracing custom_mae()')
    return tf.reduce_mean(tf.abs(y_pred - y_true))

In [190]:
class custom_dense(keras.layers.Layer): #Custom layer
    def __init__(self, units, activation = None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)
    def build(self, input_shape):
        self.kernel = self.add_weight(name = 'kernel', shape = (input_shape[1], self.units), initializer = 'uniform', trainable = True)
        self.bias = self.add_weight(name = 'bias', shape = (self.units), initializer = 'zeros', trainable = True)
        super().build(input_shape)
    def call(self, x):
        print('Tracing custom_dense()')
        return self.activation(x @ self.kernel + self.bias)

In [191]:
pre_model()

In [192]:
class custom_model(keras.models.Model): # Custom model
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.hidden_1 = custom_dense(30, activation = keras.activations.relu)
        self.hidden_2 = custom_dense(30, activation = keras.activations.relu)
        self.output_ = custom_dense(1)
    def call(self, inputs):
        print('Tracing custom_model()')
        hidden_1 = self.hidden_1(inputs)
        hidden_2 = self.hidden_2(hidden_1)
        concat = keras.layers.concatenate([inputs, hidden_2])
        output = self.output_(concat)
        return output

In [193]:
model = custom_model()

In [194]:
model.compile(loss = custom_mse, optimizer = keras.optimizers.Nadam(), metrics = [custom_mae])

In [195]:
model.fit(x_train_scaled, y_train, epochs = 5, validation_data = (x_valid_scaled, y_valid))

Epoch 1/5
Tracing custom_model()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_mse()
Tracing custom_mae()
Tracing custom_model()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_mse()
Tracing custom_mae()
345/363 [===========================>..] - ETA: 0s - loss: 1.3649 - custom_mae: 0.8048Tracing custom_model()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_mse()
Tracing custom_mae()
363/363 [==============================] - 1s 1ms/step - loss: 1.3255 - custom_mae: 0.7900 - val_loss: 0.5569 - val_custom_mae: 0.4819
Epoch 2/5
363/363 [==============================] - 0s 1ms/step - loss: 0.4419 - custom_mae: 0.4767 - val_loss: 0.4664 - val_custom_mae: 0.4576
Epoch 3/5
363/363 [==============================] - 0s 841us/step - loss: 0.4159 - custom_mae: 0.4582 - val_loss: 0.6766 - val_custom_mae: 0.4522
Epoch 4/5
363/363 [==============================] - 0s 930us/step - loss: 

- By default keras converts the python functions into TF functions automatically and we dont have to use `@tf.function`. we can turn this off by creating the model with `dynamic = True`. 

In [196]:
pre_model()

In [197]:
model = custom_model(dynamic = True)

In [198]:
model.compile(loss = custom_mse, optimizer = 'nadam', metrics = [custom_mae])

In [200]:
model.fit(x_train_scaled[:64], y_train[:64], epochs = 5, validation_data = (x_valid_scaled[:64], y_valid[:64]), verbose = 0)

Tracing custom_model()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_mse()
Tracing custom_mae()
Tracing custom_model()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_mse()
Tracing custom_mae()
Tracing custom_model()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_mse()
Tracing custom_mae()
Tracing custom_model()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_mse()
Tracing custom_mae()
Tracing custom_model()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_mse()
Tracing custom_mae()
Tracing custom_model()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_mse()
Tracing custom_mae()
Tracing custom_model()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_mse()
Tracing custom_mae()
Tracing custom_model()
Tracing custom_dense()
Tracing custom_d

- Alternatively we can compile a model with `run_eagerly = True`

In [215]:
pre_model()

In [216]:
model = custom_model()

In [217]:
model.compile(loss = custom_mse, optimizer = 'nadam', metrics = [custom_mae], run_eagerly = True)

In [218]:
model.fit(x_train_scaled[:64], y_train[:64], epochs = 5, validation_data = (x_valid_scaled[:64], y_valid[:64]), verbose = 0)

Tracing custom_model()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_mse()
Tracing custom_mae()
Tracing custom_model()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_mse()
Tracing custom_mae()
Tracing custom_model()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_mse()
Tracing custom_mae()
Tracing custom_model()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_mse()
Tracing custom_mae()
Tracing custom_model()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_mse()
Tracing custom_mae()
Tracing custom_model()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_mse()
Tracing custom_mae()
Tracing custom_model()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_dense()
Tracing custom_mse()
Tracing custom_mae()
Tracing custom_model()
Tracing custom_dense()
Tracing custom_d